In [ ]:
########################## CONDA_INSTALLATION ##########################
# !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x ./Miniconda3-latest-Linux-x86_64.sh
# !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# !rm ./Miniconda3-latest-Linux-x86_64.sh
########################################################################

In [ ]:
# !git clone https://github.com/divyjx/Video-LLaVA
# %cd Video-LLaVA

In [ ]:
########################## CONDA_ENV_CREATION ##########################
# %%shell
# eval "$(conda shell.bash hook)"
# conda create -n videollava python=3.10 -y
# conda init --all
########################################################################

In [ ]:
###########CONDA_SETUP_FOR_EACH_CELL###################
# %%shell
# eval "$(conda shell.bash hook)"
# conda activate videollava
#######################################################

!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation
!pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d
!pip install tensorboardX
!pip install requests

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id= "LanguageBind/Video-LLaVA-Pretrain-7B", local_dir = 'checkpoints/Video-LLaVA-Pretrain-7B', local_dir_use_symlinks=True)

# Loading Dataset

In [10]:
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id= "liuhaotian/LLaVA-CC3M-Pretrain-595K", local_dir = 'tuning_data', local_dir_use_symlinks=True, repo_type = 'dataset')

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id= "divyjx/VideoLLaVA_dataset", local_dir = 'tuning_data', local_dir_use_symlinks=True, repo_type = 'dataset')

In [ ]:
!ls tuning_data/merged

In [ ]:
# import zipfile
# with zipfile.ZipFile('tuning_data/images.zip', 'r') as zip_ref:
#     zip_ref.extractall('llava_image_tune')

# %cd ..
# !rm tuning_data/images.zip

In [ ]:
import torch
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

In [ ]:
# import json

# with open('tuning_data/chat.json', 'r') as file:
#     data = json.load(file)

# for entry in data:
#     entry['image'] = 'images/' + entry['image']

# with open('tuning_data/chat_new.json', 'w') as file:
#     json.dump(data, file, indent=2)

In [ ]:
# import json
# f = open('tuning_data/chat_new.json')
# data = json.load(f)
# data

In [ ]:
# !rm tuning_data/chat.json
# !rm tuning_data/metadata.json

# Edit Script
copy DATA_ROOT="image.... cell and then paste it in original file scripts/v1_5/finetune.sh

In [ ]:
# change log
# bf16 not supported => bf16 False
# tf32 not supported => tf32 False
# hugging face error => TRANSFORMERS_OFFLINE = 0
# low ram error

####### Do not run below cell  ########

In [ ]:
!nvidia-smi

# modify finetune_final.sh

In [23]:
%%writefile scripts/v1_5/finetune_final.sh

DATA_ROOT="data_root"
DATA_ROOT_IMAGE="images"
DATA_ROOT_VIDEO="videos"

HF_DATASETS_OFFLINE=1 TRANSFORMERS_OFFLINE=0 CUDA_VISIBLE_DEVICES=0,1,2,3 deepspeed llava/train/train_mem.py \
    --deepspeed ./scripts/zero2.json \
    --model_name_or_path lmsys/vicuna-7b-v1.5 \
    --version v1 \
    --data_path tuning_data/merged/merged_100.json \
    --video_folder ${DATA_ROOT_VIDEO} \
    --image_folder ${DATA_ROOT_IMAGE} \
    --X "Video" "Image" \
    --video_tower LanguageBind/LanguageBind_Video_merge \
    --image_tower LanguageBind/LanguageBind_Image \
    --pretrain_mm_mlp_adapter checkpoints/Video-LLaVA-Pretrain-7B/mm_projector.bin \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_x_start_end False \
    --mm_use_x_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir ./checkpoints/output/Video-LLaVA-7B \
    --num_train_epochs 1 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 2 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 8 \
    --lazy_preprocess True \
    --report_to tensorboard \
    --cache_dir "./cache_dir"

Overwriting scripts/v1_5/finetune_final.sh


# edit zero2.json

In [24]:
%%writefile scripts/zero2.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "bf16": {
        "enabled": "auto"
    },
    "train_micro_batch_size_per_gpu": "auto",
    "train_batch_size": "auto",
    "gradient_accumulation_steps": "auto",
    "zero_optimization": {
        "stage": 2,
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": "auto"
    }
}

Overwriting scripts/zero2.json


In [25]:
###########CONDA_SETUP_FOR_EACH_CELL###################
%%shell
eval "$(conda shell.bash hook)"
conda activate videollava
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1
#######################################################
bash scripts/v1_5/finetune_final.sh

/bin/bash: line 3: fg: no job control
[2023-12-14 16:18:36,840] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-12-14 16:18:39,161] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0: setting --include=localhost:0
[2023-12-14 16:18:39,161] [INFO] [runner.py:555:main] cmd = /usr/local/envs/videollava/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None llava/train/train_mem.py --deepspeed ./scripts/zero2.json --model_name_or_path lmsys/vicuna-7b-v1.5 --version v1 --data_path tuning_data/merged/merged_10000.json --video_folder videos --image_folder images --X Video Image --video_tower LanguageBind/LanguageBind_Video_merge --image_tower LanguageBind/LanguageBind_Image --pretrain_mm_mlp_adapter checkpoints/Video-LLaVA-Pretrain-7B/mm_projecto

CalledProcessError: ignored

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# uploading checkpoints to hf

from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/checkpoints/output",
    repo_id="divyjx/VideoLLaVA_ckpt",
    repo_type="model",
)

In [ ]:
# |-Video-LLaVA
# |--tuning_data
# |--checkpoints
# |--cache_dir
# |--images
# |--videos
# |--inference.py

In [ ]:
# pasting here for backup

In [ ]:

DATA_ROOT="data_root"
DATA_ROOT_IMAGE="images"
DATA_ROOT_VIDEO="videos"

HF_DATASETS_OFFLINE=1 TRANSFORMERS_OFFLINE=0 CUDA_VISIBLE_DEVICES=0,1,2,3 deepspeed llava/train/train_mem.py \
    --deepspeed ./scripts/zero2.json \
    --model_name_or_path lmsys/vicuna-7b-v1.5 \
    --version v1 \
    --data_path tuning_data/merged/merged_10000.json \
    --video_folder ${DATA_ROOT_VIDEO} \
    --image_folder ${DATA_ROOT_IMAGE} \
    --X "Video" "Image" \
    --video_tower LanguageBind/LanguageBind_Video_merge \
    --image_tower LanguageBind/LanguageBind_Image \
    --pretrain_mm_mlp_adapter checkpoints/Video-LLaVA-Pretrain-7B/mm_projector.bin \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_x_start_end False \
    --mm_use_x_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir ./checkpoints/output/Video-LLaVA-7B \
    --num_train_epochs 1 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 2 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 8 \
    --lazy_preprocess True \
    --report_to tensorboard \
    --cache_dir "./cache_dir"